In [0]:
# !pip install tensorflowjs

In [0]:
# !pip install -U ipython

In [3]:
# Colab library to upload files to notebook
# from google.colab import files
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
keras = tf.keras

TensorFlow 2.x selected.


In [0]:
IMG_SIZE = 299 # All images will be resized to 160x160
IMG_SHAPE=(IMG_SIZE,IMG_SIZE,3)

In [5]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [6]:
len(base_model.layers)

311

In [0]:
# Feature Extression
base_model.trainable = False

In [0]:
# # Fine Tunnig
# base_model.trainable = True
# # Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 2400

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable =  False

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)

In [0]:
model2 = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [12]:
!git clone https://github.com/sreeragrnandan/Data_Track_illegal_activities.git

Cloning into 'Data_Track_illegal_activities'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7761 (delta 0), reused 5 (delta 0), pack-reused 7754
Receiving objects: 100% (7761/7761), 2.77 GiB | 50.72 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Checking out files: 100% (6190/6190), done.


In [13]:
!ls

Data_Track_illegal_activities  sample_data


In [0]:
base_dir = './Data_Track_illegal_activities/Smoking'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training smoking pictures
train_smoking_dir = os.path.join(train_dir, 'smoking')

# Directory with our training not_smoking pictures
train_not_smoking_dir = os.path.join(train_dir, 'not_smoking')

# Directory with our validation smoking pictures
validation_smoking_dir = os.path.join(validation_dir, 'smoking')

# Directory with our validation not_smoking pictures
validation_not_smoking_dir = os.path.join(validation_dir, 'not_smoking')

In [15]:
train_smoking_fnames = os.listdir(train_smoking_dir)
print(train_smoking_fnames[:10])

train_not_smoking_fnames = os.listdir(train_not_smoking_dir)
train_not_smoking_fnames.sort()
print(train_not_smoking_fnames[:10])

['cctv27.0.jpg', 'smk_641.0.jpg', 'cctv2_835.0.jpg', 'smk_555.0.jpg', 'cctv3_450.0.jpg', 'smk_154.0.jpg', 'smk_375.0 (2).jpg', 'cctv2_43.0.jpg', 'smk_407.0 (2).jpg', 'cctv2_551.0.jpg']
['000015.jpg', '000022.jpg', '000119.jpg', '000137.jpg', '000186.jpg', '000191.jpg', '000277.jpg', '000279.jpg', '000296.jpg', '000323.jpg']


In [16]:
print('total training smoking images:', len(os.listdir(train_smoking_dir)))
print('total training not_smoking images:', len(os.listdir(train_not_smoking_dir)))
print('total validation smoking images:', len(os.listdir(validation_smoking_dir)))
print('total validation not_smoking images:', len(os.listdir(validation_not_smoking_dir)))

total training smoking images: 1504
total training not_smoking images: 1936
total validation smoking images: 914
total validation not_smoking images: 648


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [18]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [0]:
base_learning_rate = 0.0001
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [54]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(299, 299),  # All images will be resized to 150x150
        batch_size=40,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(299, 299),
        batch_size=40,
        class_mode='binary')

Found 3440 images belonging to 2 classes.
Found 1562 images belonging to 2 classes.


In [0]:
train_generator[0]

In [22]:
model2.input

<tf.Tensor 'inception_v3_input:0' shape=(None, 299, 299, 3) dtype=float32>

In [55]:
history = model2.fit_generator(
      train_generator,
      steps_per_epoch=86,  # 3440 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=39,  # 1562 images = batch_size * steps
      verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 86 steps, validate for 39 steps
Epoch 1/15


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


86/86 - 159s - loss: 0.3817 - accuracy: 0.8608 - val_loss: 4.1239 - val_accuracy: 0.6574
Epoch 2/15
86/86 - 156s - loss: 0.3990 - accuracy: 0.8465 - val_loss: 3.8811 - val_accuracy: 0.6660
Epoch 3/15
86/86 - 153s - loss: 0.3534 - accuracy: 0.8654 - val_loss: 3.8936 - val_accuracy: 0.6657
Epoch 4/15
86/86 - 153s - loss: 0.3482 - accuracy: 0.8677 - val_loss: 3.9189 - val_accuracy: 0.6689
Epoch 5/15
86/86 - 148s - loss: 0.3513 - accuracy: 0.8741 - val_loss: 3.9605 - val_accuracy: 0.6654
Epoch 6/15
86/86 - 149s - loss: 0.3367 - accuracy: 0.8731 - val_loss: 3.7531 - val_accuracy: 0.6785
Epoch 7/15
86/86 - 148s - loss: 0.3215 - accuracy: 0.8808 - val_loss: 3.9705 - val_accuracy: 0.6686
Epoch 8/15
86/86 - 149s - loss: 0.3184 - accuracy: 0.8760 - val_loss: 3.7748 - val_accuracy: 0.6785
Epoch 9/15
86/86 - 148s - loss: 0.3121 - accuracy: 0.8791 - val_loss: 3.7355 - val_accuracy: 0.6853
Epoch 10/15
86/86 - 149s - loss: 0.3095 - accuracy: 0.8828 - val_loss: 3.7768 - val_accuracy: 0.6811
Epoch 11/1

In [24]:
!rm -r ./tfjs/

rm: cannot remove './tfjs/': No such file or directory


In [0]:
# import tensorflowjs as tfjs
# tfjs_target_dir = "./tfjs"
# tfjs.converters.save_keras_model(model2, tfjs_target_dir)

In [0]:
# !ls tfjs

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img

In [0]:
import skimage.io as skio
import os
from skimage.color import rgb2gray
from skimage.transform import resize

In [35]:
!wget https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/train/smoking/cctv121.0.jpg
!wget https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/train/not_smoking/000015.jpg

--2020-02-13 03:53:06--  https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/train/smoking/cctv121.0.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 559253 (546K) [image/jpeg]
Saving to: ‘cctv121.0.jpg’

cctv121.0.jpg       100%[===================>] 546.15K  --.-KB/s    in 0.1s    

2020-02-13 03:53:06 (5.12 MB/s) - ‘cctv121.0.jpg’ saved [559253/559253]

--2020-02-13 03:53:12--  https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/train/not_smoking/000015.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
H

In [36]:
!ls

000015.jpg    cctv121.0.jpg		     sample_data
000015.jpg.1  Data_Track_illegal_activities


In [0]:
CLASS = ["Smoking","Not Smoking"]

In [33]:
# not Smoking
img=load_image("./000015.jpg")
predict = model2.predict(np.expand_dims(img,0))
print(CLASS[np.argmax(predict)])

Not Smoking


In [38]:
#Smoking
img=load_image("./cctv121.0.jpg")
predict = model2.predict(np.expand_dims(img,0))
print(CLASS[np.argmax(predict)])

Smoking


In [0]:
!wget https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/validation/smoking/cctv2_100.0.jpg
!wget https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/validation/not_smoking/cctv2_0.0.jpg

In [58]:
# Smoking on validation
img=load_image("./cctv2_0.0.jpg.1")
predict = model2.predict(np.expand_dims(img,0))
print(CLASS[np.argmax(predict)])

Not Smoking


In [0]:
#Not Smoking on validation
img=load_image("./ggg187.jpeg")
predict = model2.predict(np.expand_dims(img,0))
print(CLASS[np.argmax(predict)])

In [0]:
!wget https://raw.githubusercontent.com/sreeragrnandan/Data_Track_illegal_activities/master/Smoking/validation/not_smoking/ggg187.jpeg

In [56]:
#Not Smoking on validation
img=load_image("./ggg187.jpeg")
predict = model2.predict(np.expand_dims(img,0))
print(CLASS[np.argmax(predict)])

Not Smoking


In [0]:
# !zip -r /content/model.zip /content/tfjs/

In [0]:
# from google.colab import files
# files.download('model.zip') 